In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
import os
import json
import requests
import unicodedata

import repos 

from bs4 import BeautifulSoup

from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from time import strftime

from wordcloud import WordCloud

from env import get_connection
import acquire as ac


In [2]:
url = 'https://github.com/search?o=desc&p=3&q=shoes&s=stars&type=Repositories'

response = requests.get(url)
response

<Response [200]>

In [3]:
response.text

'\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" data-a11y-animated-images="system">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-719f1193e0c0.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-0c343b529849.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" media="all" rel="stylesheet" data-href="https:/

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
soup.find_all('a')

[<a class="px-2 py-4 color-bg-accent-emphasis color-fg-on-emphasis show-on-focus js-skip-to-content" href="#start-of-content">Skip to content</a>,
 <a aria-label="Homepage" class="mr-lg-3 color-fg-inherit flex-order-2" data-ga-click="(Logged out) Header, go to homepage, icon:logo-wordmark" href="https://github.com/">
 <svg aria-hidden="true" class="octicon octicon-mark-github" data-view-component="true" height="32" version="1.1" viewbox="0 0 16 16" width="32">
 <path d="M8 0C3.58 0 0 3.58 0 8c0 3.54 2.29 6.53 5.47 7.59.4.07.55-.17.55-.38 0-.19-.01-.82-.01-1.49-2.01.37-2.53-.49-2.69-.94-.09-.23-.48-.94-.82-1.13-.28-.15-.68-.52-.01-.53.63-.01 1.08.58 1.23.82.72 1.21 1.87.87 2.33.66.07-.52.28-.87.51-1.07-1.78-.2-3.64-.89-3.64-3.95 0-.87.31-1.59.82-2.15-.08-.2-.36-1.02.08-2.12 0 0 .67-.21 2.2.82.64-.18 1.32-.27 2-.27.68 0 1.36.09 2 .27 1.53-1.04 2.2-.82 2.2-.82.44 1.1.16 1.92.08 2.12.51.56.82 1.27.82 2.15 0 3.07-1.87 3.75-3.65 3.95.29.25.54.73.54 1.48 0 1.07-.01 1.93-.01 2.2 0 .21.15.46.55

In [6]:
soup.find_all('a')

[<a class="px-2 py-4 color-bg-accent-emphasis color-fg-on-emphasis show-on-focus js-skip-to-content" href="#start-of-content">Skip to content</a>,
 <a aria-label="Homepage" class="mr-lg-3 color-fg-inherit flex-order-2" data-ga-click="(Logged out) Header, go to homepage, icon:logo-wordmark" href="https://github.com/">
 <svg aria-hidden="true" class="octicon octicon-mark-github" data-view-component="true" height="32" version="1.1" viewbox="0 0 16 16" width="32">
 <path d="M8 0C3.58 0 0 3.58 0 8c0 3.54 2.29 6.53 5.47 7.59.4.07.55-.17.55-.38 0-.19-.01-.82-.01-1.49-2.01.37-2.53-.49-2.69-.94-.09-.23-.48-.94-.82-1.13-.28-.15-.68-.52-.01-.53.63-.01 1.08.58 1.23.82.72 1.21 1.87.87 2.33.66.07-.52.28-.87.51-1.07-1.78-.2-3.64-.89-3.64-3.95 0-.87.31-1.59.82-2.15-.08-.2-.36-1.02.08-2.12 0 0 .67-.21 2.2.82.64-.18 1.32-.27 2-.27.68 0 1.36.09 2 .27 1.53-1.04 2.2-.82 2.2-.82.44 1.1.16 1.92.08 2.12.51.56.82 1.27.82 2.15 0 3.07-1.87 3.75-3.65 3.95.29.25.54.73.54 1.48 0 1.07-.01 1.93-.01 2.2 0 .21.15.46.55

In [7]:
pageCount = 15
urlsList = []

base = 'https://github.com/search?o=desc&p={}&q=shoes&s=stars&type=Repositories' #curly braces let you format

for x in range(pageCount)[0:]:
    urlsList.append(base.format(x))

In [8]:
urlsList

['https://github.com/search?o=desc&p=0&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=1&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=2&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=3&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=4&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=5&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=6&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=7&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=8&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=9&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=10&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=11&q=shoes&s=stars&type=Repositories',
 'https://github.com/search?o=desc&p=12&q=shoes&s=stars&type=Repositories',
 'https://github.com/s

In [4]:
repo_list = ac.get_repo_links(topic='shoes', number_of_pages=5)

Page 1 Completed
Page 2 Completed
Page 3 Completed
Page 4 Completed
Page 5 Completed


In [5]:
len(repo_list)

98

In [4]:
readme_files = ac.scrape_github_data()

In [6]:
len(readme_files)

98

In [8]:
readme_files[0]

{'repo': 'shoes/shoes4',
 'language': 'Ruby',
 'readme_contents': '# Shoes 4 [![Linux Build Status](https://secure.travis-ci.org/shoes/shoes4.svg?branch=master)](http://travis-ci.org/shoes/shoes4)[![Windows Build status](https://ci.appveyor.com/api/projects/status/ejrrqefbfuhdmahj/branch/master?svg=true)](https://ci.appveyor.com/project/PragTob/shoes4/branch/master)[![Code Climate](https://codeclimate.com/github/shoes/shoes4/badges/gpa.svg)](https://codeclimate.com/github/shoes/shoes4)[![Test Coverage](https://codeclimate.com/github/shoes/shoes4/badges/coverage.svg)](https://codeclimate.com/github/shoes/shoes4/code?sort=covered_percent&sort_direction=desc)\n\nShoes 4 : the next version of Shoes\n\n## About Shoes\n\nShoes is a little DSL for cross-platform (Mac, Windows, and Linux) GUI programming. It feels like real Ruby, rather than just another C++ library wrapper. For some samples, the manual, and a free book, check out [the Shoes website](http://shoesrb.com/).\n\n## Basic Usage\n\n

In [10]:
df = pd.DataFrame(readme_files)

In [11]:
df

,repo,language,readme_contents
0,shoes/shoes4,Ruby,# Shoes 4 [![Linux Build Status](https://secur...
1,druv5319/Sneaks-API,JavaScript,"\n<p align=""center"">\n \n<img src=""https://gi..."
2,shoes/shoes3,C,\n ((( |||_| ///\ [[[_ ...
3,WebAR-rocks/WebAR.rocks.hand,JavaScript,# JavaScript/WebGL lightweight and robust hand...
4,brandontruggles/Selenium-Shoe-Bot-NakedCPH,Python,# Selenium Shoe Bot (DEPRECATED!)\n\n**This pr...
...,...,...,...
93,Kvas1407/ecomm-store-project,HTML,# Web Design & Development 1\n## E-Comm Websto...
94,bisofts/kingshoes,None,# kingshoes.-test.github.io
95,yug20/react-gltf-models-shoes,JavaScript,![Screenshot from 2022-12-26 18-19-57](https:/...
96,Kenzothd/Shoedog_Client,TypeScript,# Getting Started with Create React App\n\nThi...
